# Step 3 -  nodal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
import pandas as pd
from data import L_D_Hours, Wind_Hours, PC_DA, P_max, Load_Demand, Cost_Generation, Wind_Farm_Power, Wind, T, N, Gen, Dem, B, Node_Generator, NodeWind_Farm, Node_Demand, Line, Capacity  # Import data from data.py

# Load bid data from a CSV file
file_path = "day_ahead_bids.csv"
df = pd.read_csv(file_path)

# Ensure correct column names
df.columns = ["Hour", "Bid Price", "Bid Amount"]

# Convert bid data into a dictionary grouped by hour
bid_data = {}
for hour, group in df.groupby("Hour"):
    bid_data[hour] = group[["Bid Price", "Bid Amount"]].to_dict(orient="records")

# Create a bid price map for each demand node (1 to Dem) for each hour
bid_prices = {} 
for hour in range(1, T + 1):
    # Get the sorted bids by price for the current hour
    sorted_bids = sorted(bid_data.get(hour, []), key=lambda x: x["Bid Price"])
    
    # Allocate bid prices to demand nodes based on sorted bids
    total_bid_amount = 0
    bid_prices[hour] = []
    
    for bid in sorted_bids:
        bid_price = bid["Bid Price"]
        bid_amount = bid["Bid Amount"]
        
        # Allocate the bid price to demand nodes 
        for _ in range(int(bid_amount)): 
            bid_prices[hour].append(bid_price)
    
    if len(bid_prices[hour]) < Dem:
        # Fill the remaining demand nodes with the highest available price
        highest_price = sorted_bids[-1]["Bid Price"]
        bid_prices[hour].extend([highest_price] * (Dem - len(bid_prices[hour])))


# Define the hour for the copper plate model
h = 7  

# Model
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

# Indices
model.Gen = RangeSet(1, Gen)
model.Wind = RangeSet(1, Wind)
model.Dem = RangeSet(1, Dem)
model.N = RangeSet(1, N)

# Variables
model.P_g = Var(model.Gen, within=NonNegativeReals)  # Conventional generation
model.P_w = Var(model.Wind, within=NonNegativeReals)  # Wind farm production
model.P_d = Var(model.Dem, within=NonNegativeReals)  # Demand
model.theta = Var(model.N)  # Voltage angles

# Reference voltage at bus 1
model.theta[1].fix(0)

# Objective function
def objective_rule(model):
    return sum(bid_prices[h][dem] * model.P_d[dem] for dem in model.Dem) - \
           sum(PC_DA[gen-1] * model.P_g[gen] for gen in model.Gen) - \
           sum(0 * model.P_w[wind] for wind in model.Wind)

model.obj = Objective(rule=objective_rule, sense=maximize)

# Constraints

# Capacity constraints
model.Conventional_Generation = ConstraintList()
for gen in model.Gen:
    model.Conventional_Generation.add(model.P_g[gen] <= P_max[gen-1])

model.Wind_Generation = ConstraintList()
for wind in model.Wind:
    model.Wind_Generation.add(model.P_w[wind] <= Wind_Hours[wind-1, h-1])

# Demand constraint
model.Demand = ConstraintList()
for dem in model.Dem:
    model.Demand.add(model.P_d[dem] <= L_D_Hours[dem-1, h-1])

# Power balance constraint
model.Balance = ConstraintList()
balance_constraints = {}
for n in model.N:
    constraint = model.Balance.add(
        sum(model.P_d[dem] for dem in model.Dem if Node_Demand[dem-1] == n) -
        sum(model.P_g[gen] for gen in model.Gen if Node_Generator[gen-1] == n) -
        sum(model.P_w[wind] for wind in model.Wind if NodeWind_Farm[wind-1] == n) +
        sum(B[n-1, j-1] * Line[n-1, j-1] * (model.theta[n] - model.theta[j]) for j in model.N) -
        sum(B[j-1, n-1] * Line[j-1, n-1] * (model.theta[j] - model.theta[n]) for j in model.N) == 0
    )
    balance_constraints[n] = constraint

# Line capacity constraint
model.Line_Capacity = ConstraintList()
for i in model.N:
    for j in model.N:
        if Line[j-1, i-1] + Line[i-1, j-1] == 1:
            model.Line_Capacity.add(inequality(-Capacity[i-1, j-1], B[i-1, j-1] * (model.theta[i] - model.theta[j]), Capacity[i-1, j-1]))

# Solve
solver = SolverFactory('glpk')
result = solver.solve(model)

# Solution
if result.solver.termination_condition == TerminationCondition.optimal:
    print(f"Optimal objective value: {model.obj()} ")
    DA_price_nodal = {n: model.dual[model.Balance[n]] for n in model.N}
    pd_DA_nodal = {dem: model.P_d[dem]() for dem in model.Dem}
    pg_DA_nodal = {gen: model.P_g[gen]() for gen in model.Gen}
    pw_DA_nodal = {wind: model.P_w[wind]() for wind in model.Wind}
    theta_DA_nodal = {n: model.theta[n]() for n in model.N}

    print(f"Demand: {pd_DA_nodal}")
    print(f"Conventional generators: {pg_DA_nodal}")
    print(f"Wind Farms: {pw_DA_nodal}")
    print("Nodal Prices (LMPs):", DA_price_nodal)
    print("")
else:
    print("No optimal solution available")

Optimal objective value: 71683.94556886416 
Demand: {1: 74.4999999999999, 2: 66.7, 3: 123.6, 4: 51.0, 5: 48.9999999999999, 6: 94.1000000000003, 7: 86.3, 8: 117.7, 9: 119.600000000001, 10: 133.399999999996, 11: 182.400000000001, 12: 133.4, 13: 217.699999999996, 14: 68.6000000000026, 15: 229.499999999997, 16: 125.499999999999, 17: 88.3000000000111}
Conventional generators: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 400.0, 9: 400.000000000002, 10: 300.000000000001, 11: 146.809736799999, 12: 0.0}
Wind Farms: {1: 146.5165714, 2: 143.9089914, 3: 85.5346482, 4: 116.1434632, 5: 128.3216418, 6: 94.0649472}
Nodal Prices (LMPs): {1: 10.5199999999975, 2: 10.5199999999979, 3: 10.5199999999993, 4: 10.5200000000005, 5: 10.5199999999983, 6: 10.5199999999988, 7: 10.5199999999991, 8: 10.519999999999, 9: 10.5199999999995, 10: 10.5199999999991, 11: 10.5199999999996, 12: 10.5199999999998, 13: 10.5199999999998, 14: 10.5199999999996, 15: 10.5199999999997, 16: 10.5199999999997, 17: 10.52, 18:

# Step 3 - nodal sensitivity analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
import pandas as pd
from data import L_D_Hours, Wind_Hours, PC_DA, P_max, Load_Demand, Cost_Generation, Wind_Farm_Power, Wind, T, N_sen, Gen, Dem, B_sen, Node_Generator, NodeWind_Farm, Node_Demand, Line_sen, Capacity_sen, R_U, R_D  # Import data from data.py

# Load bid data from a CSV file
file_path = "day_ahead_bids.csv"
df = pd.read_csv(file_path)

# Ensure correct column names
df.columns = ["Hour", "Bid Price", "Bid Amount"]

# Convert bid data into a dictionary grouped by hour
bid_data = {}
for hour, group in df.groupby("Hour"):
    bid_data[hour] = group[["Bid Price", "Bid Amount"]].to_dict(orient="records")

# Create a bid price map for each demand node (1 to Dem) for each hour
bid_prices = {}  
for hour in range(1, T + 1):
    # Get the sorted bids by price for the current hour
    sorted_bids = sorted(bid_data.get(hour, []), key=lambda x: x["Bid Price"])
    
    # Allocate bid prices to demand nodes based on sorted bids
    total_bid_amount = 0
    bid_prices[hour] = []
    
    for bid in sorted_bids:
        bid_price = bid["Bid Price"]
        bid_amount = bid["Bid Amount"]
        
        # Allocate the bid price to demand nodes 
        for _ in range(int(bid_amount)):  
            bid_prices[hour].append(bid_price)
    
    if len(bid_prices[hour]) < Dem:
        # Fill the remaining demand nodes with the highest available price
        highest_price = sorted_bids[-1]["Bid Price"]
        bid_prices[hour].extend([highest_price] * (Dem - len(bid_prices[hour])))

# Define the hour for the copper plate model
h = 7  

# Model
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

# Indices
model.Gen = RangeSet(1, Gen)
model.Wind = RangeSet(1, Wind)
model.Dem = RangeSet(1, Dem)
model.N = RangeSet(1, N_sen)


# Variables
model.P_g = Var(model.Gen, within=NonNegativeReals)  # Conventional generation
model.P_w = Var(model.Wind, within=NonNegativeReals)  # Wind farm production
model.P_d = Var(model.Dem, within=NonNegativeReals)  # Demand
model.theta = Var(model.N)  # Voltage angles

# Reference voltage at bus 1
model.theta[1].fix(0)

# Objective function
def objective_rule(model):
    return sum(bid_prices[h][dem] * model.P_d[dem] for dem in model.Dem) - \
           sum(PC_DA[gen-1] * model.P_g[gen] for gen in model.Gen) - \
           sum(0 * model.P_w[wind] for wind in model.Wind)

model.obj = Objective(rule=objective_rule, sense=maximize)

# Constraints

# Capacity constraints
model.Conventional_Generation = ConstraintList()
for gen in model.Gen:
    model.Conventional_Generation.add(model.P_g[gen] <= P_max[gen-1])

model.Wind_Generation = ConstraintList()
for wind in model.Wind:
    model.Wind_Generation.add(model.P_w[wind] <= Wind_Hours[wind-1, h-1])

# Demand constraint
model.Demand = ConstraintList()
for dem in model.Dem:
    model.Demand.add(model.P_d[dem] <= L_D_Hours[dem-1, h-1])

# Power balance constraint
model.Balance = ConstraintList()
balance_constraints = {}
for n in model.N:
    constraint = model.Balance.add(
        sum(model.P_d[dem] for dem in model.Dem if Node_Demand[dem-1] == n) -
        sum(model.P_g[gen] for gen in model.Gen if Node_Generator[gen-1] == n) -
        sum(model.P_w[wind] for wind in model.Wind if NodeWind_Farm[wind-1] == n) +
        sum(B_sen[n-1, j-1] * Line_sen[n-1, j-1] * (model.theta[n] - model.theta[j]) for j in model.N) -
        sum(B_sen[j-1, n-1] * Line_sen[j-1, n-1] * (model.theta[j] - model.theta[n]) for j in model.N) == 0
    )
    balance_constraints[n] = constraint

# Line capacity constraint
model.Line_Capacity = ConstraintList()
for i in model.N:
    for j in model.N:
        if Line_sen[j-1, i-1] + Line_sen[i-1, j-1] == 1:
            model.Line_Capacity.add(inequality(-Capacity_sen[i-1, j-1], B_sen[i-1, j-1] * (model.theta[i] - model.theta[j]), Capacity_sen[i-1, j-1]))

# Solve
solver = SolverFactory('glpk')
result = solver.solve(model)

# Solution
if result.solver.termination_condition == TerminationCondition.optimal:
    print(f"Optimal objective value: {model.obj()} ")
    DA_price_nodal = {n: model.dual[model.Balance[n]] for n in model.N}
    pd_DA_nodal = {dem: model.P_d[dem]() for dem in model.Dem}
    pg_DA_nodal = {gen: model.P_g[gen]() for gen in model.Gen}
    pw_DA_nodal = {wind: model.P_w[wind]() for wind in model.Wind}
    theta_DA_nodal = {n: model.theta[n]() for n in model.N}

    print(f"Demand: {pd_DA_nodal}")
    print(f"Conventional generators: {pg_DA_nodal}")
    print(f"Wind Farms: {pw_DA_nodal}")
    print("Nodal Prices (LMPs):", DA_price_nodal)
    print("")
else:
    print("No optimal solution available")

Optimal objective value: 69697.79999418459 
Demand: {1: 74.5000000000004, 2: 66.7, 3: 123.6, 4: 51.0, 5: 48.9999999999999, 6: 94.1000000000001, 7: 86.3, 8: 117.7, 9: 119.6, 10: 133.4, 11: 182.4, 12: 133.4, 13: 217.699999999998, 14: 68.6000000000001, 15: 229.499999999998, 16: 125.5, 17: 88.3000000000002}
Conventional generators: {1: 10.8635484738216, 2: 152.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 188.283436363278, 9: 285.6627519629, 10: 300.0, 11: 309.999999999997, 12: 0.0}
Wind Farms: {1: 146.5165714, 2: 143.9089914, 3: 85.5346482, 4: 116.1434632, 5: 128.3216418, 6: 94.0649472}
Nodal Prices (LMPs): {1: 13.32, 2: 13.3660604583095, 3: 11.8540664647319, 4: 13.5052215921424, 5: 13.6240045230556, 6: 13.8024160501622, 7: 13.7791032648597, 8: 13.7791032648596, 9: 13.619136679572, 10: 13.9390698501473, 11: 15.3034125900458, 12: 13.1113352419919, 13: 13.4734308213643, 14: 18.4469818476902, 15: 9.06214969598383, 16: 7.88279339390661, 17: 6.61583333333405, 18: 6.02000000000029, 19: 9.132326

# Step 3 -  zonal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
import pandas as pd
from data import L_D_Hours, Wind_Hours, PC_DA, P_max, Load_Demand, Cost_Generation, Wind_Farm_Power, Wind, T, N, Gen, Dem, B, Node_Generator, NodeWind_Farm, Node_Demand, Line, Capacity, Z, Node_Zone, R_U, R_D, Capacity_Zone  # Import data from data.py

# Load bid data from a CSV file
file_path = "day_ahead_bids.csv"
df = pd.read_csv(file_path)

df.columns = ["Hour", "Bid Price", "Bid Amount"]

bid_data = {}
for hour, group in df.groupby("Hour"):
    bid_data[hour] = group[["Bid Price", "Bid Amount"]].to_dict(orient="records")

bid_prices = {}
for hour in range(1, T + 1):
    sorted_bids = sorted(bid_data.get(hour, []), key=lambda x: x["Bid Price"])
    bid_prices[hour] = [bid["Bid Price"] for bid in sorted_bids for _ in range(int(bid["Bid Amount"]))]
    if len(bid_prices[hour]) < Dem:
        highest_price = sorted_bids[-1]["Bid Price"]
        bid_prices[hour].extend([highest_price] * (Dem - len(bid_prices[hour])))

h = 7
divisors = [1, 2, 3, 4, 5]

for divisor in divisors:
    print(f"\nATC/{divisor}...\n")
    Capacity_Zone_adjusted = np.copy(Capacity_Zone) / divisor

    model = ConcreteModel()
    model.dual = Suffix(direction=Suffix.IMPORT)
    model.Gen = RangeSet(1, Gen)
    model.Wind = RangeSet(1, Wind)
    model.Dem = RangeSet(1, Dem)
    model.Z = RangeSet(1, Z)

    model.P_g = Var(model.Gen, within=NonNegativeReals)
    model.P_w = Var(model.Wind, within=NonNegativeReals)
    model.P_d = Var(model.Dem, within=NonNegativeReals)
    model.theta = Var(model.Z)
    model.Flow = Var(model.Z, model.Z, within=Reals)

    def objective_rule(model):
        return sum(bid_prices[h][dem] * model.P_d[dem] for dem in model.Dem) - \
               sum(PC_DA[gen-1] * model.P_g[gen] for gen in model.Gen)

    model.obj = Objective(rule=objective_rule, sense=maximize)

    model.Conventional_Generation = ConstraintList()
    for gen in model.Gen:
        model.Conventional_Generation.add(model.P_g[gen] <= P_max[gen-1])

    model.Wind_Generation = ConstraintList()
    for wind in model.Wind:
        model.Wind_Generation.add(model.P_w[wind] <= Wind_Hours[wind-1, h])

    model.Demand = ConstraintList()
    for dem in model.Dem:
        model.Demand.add(model.P_d[dem] <= L_D_Hours[dem-1, h])

    model.Balance = ConstraintList()
    for z in model.Z:
        model.Balance.add(
            sum(model.P_d[dem] for dem in model.Dem if Node_Demand[dem-1] in Node_Zone[z-1]) -
            sum(model.P_g[gen] for gen in model.Gen if Node_Generator[gen-1] in Node_Zone[z-1]) -
            sum(model.P_w[wind] for wind in model.Wind if NodeWind_Farm[wind-1] in Node_Zone[z-1]) +
            sum(model.Flow[z, j] for j in model.Z if j != z) == 0
        )

    model.Line_Capacity = ConstraintList()
    model.Definition_Power_Flow = ConstraintList()
    for i in model.Z:
        for j in model.Z:
            if i != j:
                model.Line_Capacity.add(-Capacity_Zone_adjusted[i-1, j-1] <= model.Flow[i, j])
                model.Line_Capacity.add(model.Flow[i, j] <= Capacity_Zone_adjusted[i-1, j-1])
                model.Definition_Power_Flow.add(model.Flow[i, j] == -model.Flow[j, i])

    solver = SolverFactory('glpk')
    result = solver.solve(model)

    if result.solver.termination_condition == TerminationCondition.optimal:
        print(f"Optimal objective value: {model.obj()} ")
        ZMP = {z: model.dual[model.Balance[z]] for z in model.Z}
        pg_DA_zonal = {gen: model.P_g[gen]() for gen in model.Gen}
        pw_DA_zonal = {wind: model.P_w[wind]() for wind in model.Wind}

        print("\nZonal Market Prices (ZMPs):", ZMP)
        print("\nGeneration from conventional generators:", pg_DA_zonal)
        print("\nGeneration from wind farms:", pw_DA_zonal)
        print("\nAvailable Transfer Capacities (ATC) between zones:")
        for z1 in range(Z):
            for z2 in range(Z):
                if z1 != z2:
                    print(f"From Zone {z1+1} to Zone {z2+1}: {Capacity_Zone_adjusted[z1, z2]:.2f}")
        print("")
    else:
        print(f"No optimal solution available for ATC divided by {divisor}")



ATC/1...

Optimal objective value: 83342.28042428801 

Zonal Market Prices (ZMPs): {1: 10.52, 2: 10.52, 3: 10.52}

Generation from conventional generators: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 400.0, 9: 400.0, 10: 300.0, 11: 238.8098456, 12: 0.0}

Generation from wind farms: {1: 143.1758086, 2: 146.418634, 3: 87.212025, 4: 121.150513, 5: 127.118818, 6: 103.6143558}

Available Transfer Capacities (ATC) between zones:
From Zone 1 to Zone 2: 2000.00
From Zone 1 to Zone 3: 1200.00
From Zone 2 to Zone 1: 2000.00
From Zone 2 to Zone 3: 800.00
From Zone 3 to Zone 1: 1200.00
From Zone 3 to Zone 2: 800.00


ATC/2...

Optimal objective value: 83342.28042428801 

Zonal Market Prices (ZMPs): {1: 10.52, 2: 10.52, 3: 10.52}

Generation from conventional generators: {1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 400.0, 9: 400.0, 10: 300.0, 11: 238.8098456, 12: 0.0}

Generation from wind farms: {1: 143.1758086, 2: 146.418634, 3: 87.212025, 4: 121.150513, 5: 127.118